In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def time_to_epoch(time):
    return (pd.to_datetime(time) - dt.datetime(1970,1,1)).dt.total_seconds().astype(int)


class OptionsChain():
    """
    Both calls and puts on multiple expiration dates/strikes.
    Need to add volatility smile calculations and spread calcs.
    
    """
    def __init__(self, ticker):
        
        self.ticker = ticker
        self.base_url = 'https://finance.yahoo.com/quote/' + self.ticker + '/options?' + 'c=' + self.ticker
        
        r = requests.get(self.base_url)
        
        self.data = None
        
    def get_urls(self, weeks=3):
        
        
        starting_date = 1561075200 # june 21st: will fix this
        
        urls = []
        
        for i in range(0, weeks):
            
            time = starting_date + (i * 604800)
            puts = 'https://finance.yahoo.com/quote/' + self.ticker + '/options?' + 'date=' + str(time) + '&p=' + self.ticker
            #calls = 'https://finance.yahoo.com/quote/' + self.ticker + '/options?' + 'date=' + str(time) + '&c=' + self.ticker
            urls.append(puts)
            #self.urls.append(calls)
            
        return urls
        
    def get_options_data(self, weeks=3):
        
        r = requests.get(self.base_url)
        self.data = pd.read_html(r.text)[0]
        self.data = self.data.append(pd.read_html(r.text)[1])
        
        urls = self.get_urls()
        count = 0
        for url in urls:
            r = requests.get(url)
            d = pd.read_html(r.text)[0]
            d = d.append(pd.read_html(r.text)[1])
            self.data = self.data.append(d)
            count += 1
            print(str(count) + '/' + str(weeks) + ' completed!')
    
    def remove_illiquid(self, data):
        pass
    
    def volSmile(self, data):
        pass
    
    def delta(self, data):
        pass
    
    def gamma(self, data):
        pass
    
    def vega(self, data):
        pass
    
    def impliedVol(self, data):
        pass
    
    
    
class TrendingOptions():
    """
    Options with open interest over 10k. 
    """
    
    def __init__(self):
        
        self.base_url = 'https://finance.yahoo.com/options/highest-open-interest?'
        self.data = pd.read_html(requests.get(self.base_url).text)[0]
        self.last_updated = None
        self.max_OpenInt = None
        self.min_OpenInt = None
        
    def GetTrending(self, verbose=True):
        
        
        offset = 100
        count = 100
        
        while offset + count < 3000:
            self.data = self.data.append(pd.read_html(requests.get(self.base_url + str(offset) + '=0&' + 'count=100').text)[0])
            offset += 100
            print('+100 more added - ' + str(3000 - offset) + ' to go')
            
        self.data['money-weighted-volume'] = self.data.Volume * self.data['Price (Intraday)'] * 100
        return self.data

In [3]:
options = OptionsChain('TSLA')

In [4]:
options.get_options_data()

1/3 completed!
2/3 completed!
3/3 completed!


In [8]:
# this is all non-OTC tesla options up to July 5th 2019 expiry.
# longname will be useful for updating only the quotes that are useful.
# will refactor the code was just trying to get a baseline down. 

options.data.sample(10)

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
24,TSLA190628C00222500,2019-06-17 3:59PM EDT,222.5,10.75,0.0,0.0,0.0,-,458,396,0.00%
14,TSLA190621P00085000,2019-06-14 3:59PM EDT,85.0,0.01,0.0,0.0,0.0,-,152,2001,50.00%
35,TSLA190621P00190000,2019-06-17 3:59PM EDT,190.0,0.18,0.0,0.0,0.0,-,2216,8317,50.00%
41,TSLA190628C00265000,2019-06-17 3:55PM EDT,265.0,0.60,0.0,0.0,0.0,-,129,229,12.50%
3,TSLA190621C00040000,2019-06-07 11:06AM EDT,40.0,227.20,0.0,0.0,0.0,-,10,20,0.00%
41,TSLA190621P00212500,2019-06-17 3:59PM EDT,212.5,1.82,0.0,0.0,0.0,-,2514,1504,25.00%
38,TSLA190621C00217500,2019-06-17 3:59PM EDT,217.5,10.60,0.0,0.0,0.0,-,1209,1368,0.00%
49,TSLA190621C00245000,2019-06-17 3:59PM EDT,245.0,0.68,0.0,0.0,0.0,-,1872,3742,12.50%
27,TSLA190628P00195000,2019-06-17 3:59PM EDT,195.0,1.47,0.0,0.0,0.0,-,1611,1669,25.00%
28,TSLA190621P00155000,2019-06-17 2:42PM EDT,155.0,0.02,0.0,0.0,0.0,-,119,1819,50.00%


In [17]:
trending = TrendingOptions()

In [18]:
trending.data.head()

,Symbol,Underlying Symbol,Name,Strike,Expiration Date,Price (Intraday),Change,% Change,Bid,Ask,Volume,Open Interest
0,EWZ,EWZ,EWZ Jun 2019 44.000 call,44.0,2019-06-20,0.08,0.04,+100.00%,0.07,0.09,232,351549
1,^VIX,^VIX,I:VIX Jun 2019 25.000 call,25.0,2019-06-18,0.05,0.00,0.00%,0.00,0.05,300,310332
2,CHK,CHK,CHK Jan 2020 7.000 call,7.0,2020-01-16,0.03,0.00,0.00%,0.02,0.03,10,305871
3,GE,GE,GE Jan 2020 20.000 call(CONTRAC,20.0,2020-01-16,0.01,0.00,0.00%,0.01,0.02,28,281003
4,CLDR,CLDR,CLDR Jun 2021 5.000 put,5.0,2021-06-17,1.15,0.00,0.00%,0.85,1.20,20,249008


In [19]:
trending.GetTrending()

+100 more added - 2800 to go
+100 more added - 2700 to go
+100 more added - 2600 to go
+100 more added - 2500 to go
+100 more added - 2400 to go
+100 more added - 2300 to go
+100 more added - 2200 to go
+100 more added - 2100 to go
+100 more added - 2000 to go
+100 more added - 1900 to go
+100 more added - 1800 to go
+100 more added - 1700 to go
+100 more added - 1600 to go
+100 more added - 1500 to go
+100 more added - 1400 to go
+100 more added - 1300 to go
+100 more added - 1200 to go
+100 more added - 1100 to go
+100 more added - 1000 to go
+100 more added - 900 to go
+100 more added - 800 to go
+100 more added - 700 to go
+100 more added - 600 to go
+100 more added - 500 to go
+100 more added - 400 to go
+100 more added - 300 to go
+100 more added - 200 to go
+100 more added - 100 to go


,Symbol,Underlying Symbol,Name,Strike,Expiration Date,Price (Intraday),Change,% Change,Bid,Ask,Volume,Open Interest,money-weighted-volume
0,EWZ,EWZ,EWZ Jun 2019 44.000 call,44.0,2019-06-20,0.08,0.04,+100.00%,0.07,0.09,232,351549,1856.0
1,^VIX,^VIX,I:VIX Jun 2019 25.000 call,25.0,2019-06-18,0.05,0.00,0.00%,0.00,0.05,300,310332,1500.0
2,CHK,CHK,CHK Jan 2020 7.000 call,7.0,2020-01-16,0.03,0.00,0.00%,0.02,0.03,10,305871,30.0
3,GE,GE,GE Jan 2020 20.000 call(CONTRAC,20.0,2020-01-16,0.01,0.00,0.00%,0.01,0.02,28,281003,28.0
4,CLDR,CLDR,CLDR Jun 2021 5.000 put,5.0,2021-06-17,1.15,0.00,0.00%,0.85,1.20,20,249008,2300.0
5,EEM,EEM,EEM Jun 2019 45.000 call,45.0,2019-06-20,0.01,0.00,0.00%,0.00,0.01,613,244068,613.0
6,EEM,EEM,EEM Sep 2019 36.000 put,36.0,2019-09-19,0.34,0.00,0.00%,0.25,0.27,3508,241822,119272.0
7,XLF,XLF,XLF Jun 2019 24.000 put,24.0,2019-06-20,0.01,0.00,0.00%,0.00,0.01,5,224972,5.0
8,XOP,XOP,XOP Sep 2019 25.000 put,25.0,2019-09-19,1.36,-0.19,-12.26%,1.36,1.38,496,220795,67456.0
9,EEM,EEM,EEM Jun 2019 38.000 put,38.0,2019-06-20,0.07,0.05,+250.00%,0.01,0.02,1,216921,7.0


In [ ]:
# next we need to add 